In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
#Carga del dataset

In [37]:
df30 = pd.read_csv("Estadisticas_normales_Datos_abiertos_1991-2020.txt", skiprows=7,sep="\t",encoding="latin1") #skiprows=7 elimina las primeras siete líneas (contienen texto)
df30

,Estación,Valor Medio de,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Unnamed: 14
0,LA QUIACA OBSERVATORIO,Temperatura (°C),13.2,13.0,12.8,11.3,7.3,4.8,4.5,7.0,10.0,12.4,13.4,13.9,NaN
1,LA QUIACA OBSERVATORIO,Temperatura máxima (°C),20.6,20.4,20.6,20.3,17.8,16.3,16.1,18.0,20.0,21.7,22.5,22.2,NaN
2,LA QUIACA OBSERVATORIO,Temperatura mínima (°C),7.7,7.6,6.6,3.1,-2.5,-5.7,-6.2,-4.0,-0.4,3.3,5.5,7.3,NaN
3,LA QUIACA OBSERVATORIO,Humedad relativa (%),62.6,63.2,60.3,46.0,32.6,27.4,25.7,26.7,32.1,42.4,48.6,55.8,NaN
4,LA QUIACA OBSERVATORIO,Velocidad del Viento (km/h) (2011-2020),6.5,6.8,6.7,5.5,4.8,5.5,5.9,6.7,7.9,7.9,7.7,7.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,BASE SAN MARTÍN,Humedad relativa (%),71.2,74.6,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,73.2,NaN
780,BASE SAN MARTÍN,Velocidad del Viento (km/h) (2011-2020),S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,NaN
781,BASE SAN MARTÍN,Nubosidad total (octavos),6.3,6.5,6.4,6.1,5.9,5.2,5.4,5.8,6.2,6.5,6.3,6.2,NaN
782,BASE SAN MARTÍN,Precipitación (mm),S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,NaN


In [39]:
# Eliminación de columna con datos erróneos

In [41]:
df30 = df30.drop(columns=["Unnamed: 14"])
df30

,Estación,Valor Medio de,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,LA QUIACA OBSERVATORIO,Temperatura (°C),13.2,13.0,12.8,11.3,7.3,4.8,4.5,7.0,10.0,12.4,13.4,13.9
1,LA QUIACA OBSERVATORIO,Temperatura máxima (°C),20.6,20.4,20.6,20.3,17.8,16.3,16.1,18.0,20.0,21.7,22.5,22.2
2,LA QUIACA OBSERVATORIO,Temperatura mínima (°C),7.7,7.6,6.6,3.1,-2.5,-5.7,-6.2,-4.0,-0.4,3.3,5.5,7.3
3,LA QUIACA OBSERVATORIO,Humedad relativa (%),62.6,63.2,60.3,46.0,32.6,27.4,25.7,26.7,32.1,42.4,48.6,55.8
4,LA QUIACA OBSERVATORIO,Velocidad del Viento (km/h) (2011-2020),6.5,6.8,6.7,5.5,4.8,5.5,5.9,6.7,7.9,7.9,7.7,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,BASE SAN MARTÍN,Humedad relativa (%),71.2,74.6,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,73.2
780,BASE SAN MARTÍN,Velocidad del Viento (km/h) (2011-2020),S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D
781,BASE SAN MARTÍN,Nubosidad total (octavos),6.3,6.5,6.4,6.1,5.9,5.2,5.4,5.8,6.2,6.5,6.3,6.2
782,BASE SAN MARTÍN,Precipitación (mm),S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D,S/D


In [ ]:
#Caracterización del dataframe

In [53]:
print(df30.shape) # cantidad de filas y columnas respectivamente

print(df30.columns) 

(784, 14)
Index(['Estación', 'Valor Medio de', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
       'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'],
      dtype='object')


In [ ]:
# Primeras 30 filas

In [61]:
print(df30.head(30))

                  Estación                                     Valor Medio de  \
0   LA QUIACA OBSERVATORIO                                   Temperatura (°C)   
1   LA QUIACA OBSERVATORIO                            Temperatura máxima (°C)   
2   LA QUIACA OBSERVATORIO                            Temperatura mínima (°C)   
3   LA QUIACA OBSERVATORIO                               Humedad relativa (%)   
4   LA QUIACA OBSERVATORIO            Velocidad del Viento (km/h) (2011-2020)   
5   LA QUIACA OBSERVATORIO                          Nubosidad total (octavos)   
6   LA QUIACA OBSERVATORIO                                 Precipitación (mm)   
7   LA QUIACA OBSERVATORIO  Frecuencia de días con Precipitación superior ...   
8               ORÁN AERO*                                   Temperatura (°C)   
9               ORÁN AERO*                            Temperatura máxima (°C)   
10              ORÁN AERO*                            Temperatura mínima (°C)   
11              ORÁN AERO*  

In [ ]:
# Cantidad estaciones meteorológicas

In [63]:
cantidad_estaciones = df30["Estación"].nunique()
print(cantidad_estaciones)

98


In [ ]:
# Listado de estaciones distintas

In [65]:
estaciones_distintas = df30["Estación"].unique()
print(estaciones_distintas)

['LA QUIACA OBSERVATORIO' 'ORÁN AERO*' 'TARTAGAL AERO' 'JUJUY AERO'
 'JUJUY U N' 'SALTA AERO' 'LAS LOMITAS' 'IGUAZÚ AERO' 'TUCUMÁN AERO'
 'SANTIAGO DEL ESTERO AERO' 'PRESIDENCIA ROQUE SAENZ PEÑA AERO*'
 'RESISTENCIA AERO' 'FORMOSA AERO' 'BERNARDO DE IRIGOYEN AERO'
 'CORRIENTES AERO' 'POSADAS AERO' 'OBERÁ' 'TINOGASTA' 'CHILECITO AERO'
 'LA RIOJA AERO' 'CATAMARCA AERO' 'VILLA DE MARÍA DEL RÍO SECO'
 'CERES AERO' 'RECONQUISTA AERO' 'PASO DE LOS LIBRES AERO' 'JACHAL'
 'SAN JUAN AERO' 'CHAMICAL AERO' 'VILLA DOLORES AERO' 'CÓRDOBA AERO'
 'CÓRDOBA OBSERVATORIO' 'PILAR OBSERVATORIO' 'SAUCE VIEJO AERO'
 'PARANÁ AERO' 'MONTE CASEROS AERO' 'CONCORDIA AERO' 'SAN MARTÍN (MZA)'
 'MENDOZA AERO' 'MENDOZA OBSERVATORIO' 'SAN LUIS AERO'
 'VILLA REYNOLDS AERO' 'RÍO CUARTO AERO' 'MARCOS JUÁREZ AERO'
 'VENADO TUERTO AERO' 'EL TRÉBOL' 'ROSARIO AERO' 'GUALEGUAYCHÚ AERO'
 'MALARGÜE AERO' 'SAN RAFAEL AERO' 'GENERAL PICO AERO' 'LABOULAYE AERO'
 'PEHUAJÓ AERO' 'JUNÍN AERO' 'NUEVE DE JULIO' 'SAN FERNANDO AERO*'
 '

In [ ]:
# Tipos de parámetros indicados en la tabla para cada estación meteorológica (descriptos en la columna "Valor Medio de")

In [71]:
parametros_medidos = df30["Valor Medio de"].unique()
print(parametros_medidos)

['Temperatura (°C)' 'Temperatura máxima (°C)' 'Temperatura mínima (°C)'
 'Humedad relativa (%)' 'Velocidad del Viento (km/h) (2011-2020)'
 'Nubosidad total (octavos)' 'Precipitación (mm)'
 'Frecuencia de días con Precipitación superior a 1.0 mm']


In [55]:
#**Exploración de datos**
#Datos faltantes, datos duplicados, tipos de datos

In [57]:
print(f"Valores faltantes: {df30.isnull().sum()}")

print(f"Datos duplicados: ", df30.duplicated().sum())

print(f"Columnas del dataframe:\n{df30.dtypes}\n")

Valores faltantes: Estación          0
Valor Medio de    0
Ene               0
Feb               0
Mar               0
Abr               0
May               0
Jun               0
Jul               0
Ago               0
Sep               0
Oct               0
Nov               0
Dic               0
dtype: int64
Datos duplicados:  0
Columnas del dataframe:
Estación          object
Valor Medio de    object
Ene               object
Feb               object
Mar               object
Abr               object
May               object
Jun               object
Jul               object
Ago               object
Sep               object
Oct               object
Nov               object
Dic               object
dtype: object



In [ ]:
# Cantidad de datos declarados como S/D

In [81]:
cantidad_sd = (df30 == "S/D").sum().sum()
print(cantidad_sd)

853


In [ ]:
#Estudio de datos anuales

#Media de todos los datos  a nivel país

#Estadísticas de la temperatura media

In [ ]:
Datos_temp_media = df30[df30["Valor Medio de"] == "Temperatura (°C)"].copy
print(Datos_temp_media)

meses = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", 
         "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]

In [ ]:
#Selección de los datos a estudiar - Datos de Aeroparque (Buenos Aires)
#Los guardo en df_aero

df_aero = df30[df30["Estación"] == "AEROPARQUE AERO"].copy()
print(df_aero)


#Gráficos

meses = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", 
         "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]

print(df_aero[meses].iloc[0])

print(df_aero.loc[472, meses])

def limpiar_columnas_numericas(df_aero, columnas):
    for col in columnas:
        df_aero.loc[:, col] = (
            df_aero[col]
            .astype(str)
            .str.strip()
            .str.replace(",", ".", regex=False)
            .str.replace("-", "-", regex=False)
            .str.replace("--", "", regex=False)
            .str.replace("s/d", "", case=False, regex=False)
        )
        df_aero[col] = pd.to_numeric(df_aero[col], errors='coerce')

limpiar_columnas_numericas(df_aero, meses)

print(df_aero[meses].iloc[0])

print(df_aero.loc[472, meses])





#Defino como agrupo los gráficos

grupo_1 = ["Temperatura (°C)", "Temperatura máxima (°C)", "Temperatura mínima (°C)"]
grupo_2 = ["Humedad relativa (%)", "Precipitación (mm)", "Velocidad del viento (km/h)"]
grupo_3 = ["Nubosidad (octavos)", "Frecuencia de días con Precipitación superior a 1 mm"]

print(df_aero["Valor Medio de"].unique())

#Gráfico de temperatura a lo largo del año

#Guardado de imágenes
import os
# Crear carpeta "imagenes" si no existe
os.makedirs("imagenes", exist_ok=True)

#Gráfico   
def graficar_grupo(df_aero, variables, titulo, ylim=None, guardar=False, nombre_archivo="grafico.png"):
    plt.figure(figsize=(10, 5))
    plt.figure(figsize=(10, 5))
       
    for variable in variables:
        fila = df_aero[df_aero["Valor Medio de"] == variable]
        if not fila.empty:
            plt.plot(meses, fila[meses].values.flatten(), marker='o', label=variable)

    plt.title(titulo)
    plt.xlabel("Mes")
    plt.ylabel("Valor")
    if ylim:
        plt.ylim(ylim)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    #if guardar:
    #    ruta_completa = os.path.join("imagenes", nombre_archivo)
    #    plt.savefig(ruta_completa, dpi=300)
    #    print(f"Gráfico guardado en: {ruta_completa}")
    plt.show()

# Llamadas a la función para guardar los tres gráficos
graficar_grupo(df_aero, grupo_1, "Temperaturas - Aeroparque", ylim=(0, 30),
               guardar=True, nombre_archivo="temperaturas_aeroparque.png")

graficar_grupo(df_aero, grupo_2, "Humedad, Precipitación y Viento - Aeroparque",
               guardar=True, nombre_archivo="humedad_precipitacion_viento.png")

graficar_grupo(df_aero, grupo_3, "Nubosidad y Días con Lluvia - Aeroparque",
               guardar=True, nombre_archivo="nubosidad_dias_lluvia.png")
"""